# Frequency-domain ODMR measurement (with APD & Time Tagger)



![frequency-shceduling](./figure/frequency-shceduling.jpg)


Import necessary modules and functions firstly

In [ ]:
import matplotlib.pyplot as plt
from odmactor.scheduler import CWScheduler, PulseScheduler
import scipy.constants as C
import numpy as np
import time
from odmactor.instrument import *



## CW ODMR

**1. Set measurement parameters**
- Time bin $t$ per readout operation, number of readout operation $N$ per frequency measurement point
- MW power $p$, MW frequencies for scanning modes, i.e., `[freq_start, freq_end, freq_step]`, unit: "Hz"


In [ ]:
f = 50  #unit: Hz
period = 1 / f / C.nano
N = int(1e3 / 5)

freq_start = 0.04 * C.giga
freq_end = 0.1 * C.giga
freq_step = 2 * C.mega
p = -15


**2. Construct a Continous-wave Scheduler and run**


In [ ]:
cw_scheduler = CWScheduler(with_ref=False, use_lockin=True, epoch_omit=5)

In [ ]:
cw_scheduler.reconnect()
cw_scheduler.configure_mw_paras(p)
cw_scheduler.configure_lockin_counting()
cw_scheduler.configure_odmr_seq(period, N)
cw_scheduler.set_mw_freqs(freq_start, freq_end, freq_step)

cw_scheduler.run_scanning()
cw_scheduler.close()

In [ ]:
# visualize ASG sequences
fig = cw_scheduler.sequences_figure


**3. Calculate contrast and plot figures**

In [ ]:
## count & contrast
cw_sig_ref = cw_scheduler.result  # [freqs, counts, counts_ref]
if len(cw_sig_ref) == 3:
    contrast_cw = [sig / ref for sig, ref in zip(cw_sig_ref[1], cw_sig_ref[1])]
else:
    contrast_cw = []

In [ ]:
# count
plt.plot(cw_sig_ref[0][0:], cw_sig_ref[1][0:], 'o-', label='MW on')
if len(cw_sig_ref) == 3:
    plt.plot(cw_sig_ref[0][0:], cw_sig_ref[2][0:], 'o-', label='MW off')
plt.legend()
plt.title('Counts')
plt.show()

In [ ]:
# contrast
if contrast_cw:
    plt.plot(cw_sig_ref[0][3:], contrast_cw[3:], 'o-')
    plt.title('Contrast')
    plt.show()

**4. Save data result**

In [ ]:
np.savetxt('cw-counting--{}dBm_12.txt'.format(p), np.array(cw_sig_ref))
np.savetxt('cw-contrast--{}dBm_12.txt'.format(p), np.vstack([cw_sig_ref[0], contrast_cw]).T)


## Pulse ODMR


**1. Set measurement parameters**
- number of readout operation $N$ per frequency measurement point
- MW power $p$, MW frequencies for scanning modes, i.e., `[freq_start, freq_end, freq_step]`, unit: "Hz"
- laser initilization time $t_{init}$, MW operation time $t_{mw}$, singal readout pulse time $t_{sig}$
- optional default parameters: reference signals readout time $t_{ref}$ ($t_{ref} = t_{sig}$ actually), some time interval parameters
- Time bin $t$ per readout operation is then calculated by the sequences parameters

In [ ]:
freq_start = 0.01 * C.giga
freq_end = 0.2 * C.giga
freq_step = 2 * C.mega

t_init = 3e3
t_mw = 1000

inter_init_mw = 1000
inter_mw_read = 1000

inter_readout = 0
t_read_sig = 700
pre_read = 0

N = int(1e6)

p = -15


**2. Construct a Pulse ODMR Scheduler and run**

In [ ]:
scheduler = PulseScheduler(mw_ttl=1, with_ref=False, use_lockin=True, epoch_omit=5)
scheduler.configure_mw_paras(power=p)

# single-signal readout
scheduler.configure_lockin_counting()
scheduler.configure_odmr_seq(t_init, t_mw, t_read_sig=t_read_sig, inter_init_mw=inter_init_mw,
                             pre_read=pre_read, inter_mw_read=inter_mw_read, inter_readout=inter_readout, N=N)
scheduler.set_mw_freqs(freq_start, freq_end, freq_step)

In [ ]:
# visualize sequences
fig = scheduler.sequences_figure


In [ ]:
scheduler.run_scanning()

scheduler.close()


**3. Calculate contrast and plot figures**

In [ ]:
## count & contrast
pulse_sig_ref = scheduler.result  # [freqs, counts, counts_ref]

if len(pulse_sig_ref) == 3:
    contrast_pulse = [sig / ref for sig, ref in zip(pulse_sig_ref[1], pulse_sig_ref[2])]
else:
    contrast_pulse = []

In [ ]:
plt.plot(pulse_sig_ref[0][1:], pulse_sig_ref[1][1:], 'o--', label='Signal')
if len(pulse_sig_ref) == 3:
    plt.plot(pulse_sig_ref[0][1:], pulse_sig_ref[2][1:], 'o-', label='Reference')
plt.legend()
plt.title('Counts')
plt.show()


In [ ]:
if contrast_pulse:
    plt.plot(pulse_sig_ref[0][1:], contrast_cw[1:], 'o-')
    plt.title('Pulse ODMR Contrast')
    plt.show()

**4. Save results**

In [ ]:
np.savetxt('pulse_12.txt', scheduler.result)
